In [1]:
import nltk

# nltk.download('stopwords')

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [3]:
# Gensim (Topic Modeling Pacakge)
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
# spacy for lemmatization
import spacy

from spacy_download import load_spacy

In [5]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [8]:
'''
We will be using the 20-Newsgroups dataset for this exercise.
This version of the dataset contains about 11k newsgroups posts from 
20 different topics. This is available as newsgroups.json.
'''
# Import Dataset
df = pd.read_csv('data/during1_covidTweets_v2.csv')
print(df.username.unique())
df.head()
df.count()

['AndyVermaut' 'TrendzNewsbd' 'COVIDLive' 'AhmadNorMaulana'
 'islandofcovid' 'suhail1mirza' 'yasu_yasuno_sa' 'eeeer71' 'nicktompan'
 'BenHssinAhmed2' 'Mott_Mason' 'newsfilterio' 'GlobalPandemics'
 'AquariusOnFire' 'VonPyotr' 'SharonShepshan' 'tazalot1981' 'latimes'
 'Ravie777' 'schisano85' 'morning_news24h' 'CTVNews' 'unstoppable8672'
 'boatratbown' 'Leedslad48' 'jlwoods0241' 'merapimanf' 'BobinSea'
 'renztamayo' 'durindale' 'Hibatu3' 'alicuff' 'CGJr_carlos' 'yahata11'
 'LouWasHe' 'andy68101744' 'mlauriat' 'DefaultNu' 'TraderMarcoCost'
 'VivekSubbiah' 'meer10001' 'jonathanrockoff' 'JerrBoi' 'Alan_Nishihara'
 'NuritBaytch' 'NZStuff' 'elvisjj' 'WatchOurCity' 'onelinernews123'
 'CassieY4' 'SetTheoryVirus' 'ZyiteGadgets' 'bmangh' 'roto_rua'
 'jayjames4' 'usatodaysun21' 'chuck85258' 'tjmakiboi' 'Su15100307'
 'Melodys_Musings' 'jebender1' 'Reuters' 'caa1000' 'DennisKendel'
 'TweeterSomebody' 'Nesgirl914' 'Nan10' 'thedextazlab' 'fox5dc' 'EGirald'
 'MelanieMoore' 'easyOntario' 'JaxRaymond' 'Er

datetime          147389
tweet_id          147389
text              147389
username          147389
url               147389
user_location     117081
like_count        147389
retweet_count     147389
follower_count    147389
reply_count       147389
verified          147389
hashtags           27728
dtype: int64

In [9]:
'''
As you can see there are many emails, newline and extra spaces that is quite distracting.
Let’s get rid of them using regular expressions.
'''
# Convert to list
data = df.text.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

#remove URL
data = [re.sub('http://\S+|https://\S+', '', sent) for sent in data]
data = [re.sub('http[s]?://\S+', '', sent) for sent in data]
data = [re.sub(r"http\S+", "", sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['US Authorizes Johnson &amp; Johnson Covid Vaccine For Emergency Use ']


In [10]:

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['us', 'authorizes', 'johnson', 'amp', 'johnson', 'covid', 'vaccine', 'for', 'emergency', 'use']]


In [11]:

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['us', 'authorizes', 'johnson', 'amp', 'johnson', 'covid', 'vaccine', 'for', 'emergency_use']


In [12]:

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Will download the model if it isn't installed yet 
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



print(data_lemmatized[:1])

[['authorize', 'emergency']]


In [14]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1)]]


In [15]:
vocab = len(id2word.token2id)

pd.Series([t for tokens in data_words_bigrams for t in tokens if "_" in t]).unique()

array(['fda_authorizes', 'thus_far', 'worrying_six', ...,
       'whooping_cough', 'polio_diphtheria', 'infections_ezekiel'],
      dtype=object)

In [16]:

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]



[(0,
  '0.068*"year" + 0.042*"country" + 0.024*"number" + 0.024*"home" + '
  '0.022*"roll" + 0.021*"come" + 0.021*"still" + 0.020*"month" + 0.020*"learn" '
  '+ 0.019*"last"'),
 (1,
  '0.211*"covid" + 0.131*"vaccine" + 0.034*"say" + 0.026*"get" + '
  '0.018*"public" + 0.017*"ecosearch" + 0.015*"need" + 0.013*"time" + '
  '0.013*"know" + 0.012*"work"'),
 (2,
  '0.101*"vaccination" + 0.080*"see" + 0.070*"week" + 0.066*"hospital" + '
  '0.065*"care" + 0.063*"update" + 0.047*"expert" + 0.043*"staff" + '
  '0.034*"medical" + 0.034*"early"'),
 (3,
  '0.188*"new" + 0.159*"death" + 0.121*"case" + 0.096*"die" + 0.082*"report" + '
  '0.047*"record" + 0.039*"kill" + 0.026*"rise" + 0.015*"daily" + 0.012*"man"'),
 (4,
  '0.070*"coronavirus" + 0.065*"health" + 0.048*"people" + 0.042*"virus" + '
  '0.038*"amp" + 0.033*"pfizer" + 0.025*"make" + 0.024*"life" + 0.022*"dose" + '
  '0.018*"spread"'),
 (5,
  '0.157*"pandemic" + 0.060*"many" + 0.035*"key" + 0.031*"find" + '
  '0.026*"thing" + 0.025*"today" 

In [18]:
'''
Model perplexity and topic coherence provide a convenient measure to judge 
how good a given topic model is. In my experience, topic coherence score, 
in particular, has been more helpful.
'''
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)




Perplexity:  -8.371148794039476

Coherence Score:  0.29347907960219227


In [19]:
'''
Now that the LDA model is built, the next step is to examine the produced 
topics and the associated keywords. There is no better tool than pyLDAvis package’s 
interactive chart and is designed to work well with jupyter notebooks.
'''
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\hannah.kim\AppData\Roaming\Python\Python39\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.299200  0.234338       1        1  31.245957
4      0.193909 -0.004451       2        1  15.000729
0      0.056643 -0.326259       3        1  10.314153
8      0.039454 -0.139328       4        1   8.940079
7      0.049956  0.039887       5        1   8.073581
3     -0.197346  0.128902       6        1   5.956402
9     -0.029712 -0.123682       7        1   5.435008
6     -0.082207  0.074513       8        1   5.402323
5     -0.174518  0.070062       9        1   5.262860
2     -0.155380  0.046018      10        1   4.368909, topic_info=             Term          Freq         Total Category  logprob  loglift
9           covid  82790.000000  82790.000000  Default  30.0000  30.0000
39        vaccine  51359.000000  51359.000000  Default  29.0000  29.0000
5             new  14035.000000  14035.000000  Default  28.0000  28.0000
4           death  11869.000000  11869.000000  Default  27.0000  27.0000
22       pandemic  10358.000000  10358.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
12991     biology    590.178014    592.220912  Topic10  -4.5314   3.1272
258      position    209.650439    210.382746  Topic10  -5.5664   3.1272
159     regulator    209.127652    209.859679  Topic10  -5.5689   3.1272
4051        union    196.756591    197.488689  Topic10  -5.6298   3.1269
25081  roy_cooper    256.678536    258.803609  Topic10  -5.3640   3.1224

[344 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1564      8  0.998669          accord
268      10  0.998840      additional
141       8  0.998658  administration
1080      5  0.999097           ahead
165       9  0.998488             aid
...     ...       ...             ...
820       5  0.999871           world
78        3  0.999942            year
421       7  0.999759             yet
669       7  0.998558           young
6468     10  0.998718         youtube

[368 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 1, 9, 8, 4, 10, 7, 6, 3])

In [20]:
output = lda_model.show_topics(num_words=30)

In [21]:
output_df = pd.DataFrame()
for o in output:    
    d = {}
    topic_num = o[0]
    distribution = o[1]
    decimal_list = re.findall(r'\d+\.\d+', distribution)
    word_list = re.findall(r'"([^"]*)"', distribution)
    d["topic_" + str(topic_num + 1) + "_word"] = word_list
    d["topic_" + str(topic_num + 1) + "_dist"] =  decimal_list

    topic_df = pd.DataFrame(d)
    output_df = pd.concat([output_df, topic_df], axis=1)

output_df

,topic_1_word,topic_1_dist,topic_2_word,topic_2_dist,topic_3_word,topic_3_dist,topic_4_word,topic_4_dist,topic_5_word,topic_5_dist,topic_6_word,topic_6_dist,topic_7_word,topic_7_dist,topic_8_word,topic_8_dist,topic_9_word,topic_9_dist,topic_10_word,topic_10_dist
0,year,0.068,covid,0.211,vaccination,0.101,new,0.188,coronavirus,0.070,pandemic,0.157,give,0.067,world,0.075,restriction,0.035,day,0.118
1,country,0.042,vaccine,0.131,see,0.080,death,0.159,health,0.065,many,0.060,late,0.057,first,0.066,go,0.033,hope,0.045
2,number,0.024,say,0.034,week,0.070,case,0.121,people,0.048,key,0.035,biontech,0.044,take,0.053,start,0.029,receive,0.044
3,home,0.024,get,0.026,hospital,0.066,die,0.096,virus,0.042,find,0.031,datum,0.032,test,0.044,population,0.028,plan,0.035
4,roll,0.022,public,0.018,care,0.065,report,0.082,amp,0.038,thing,0.026,shot,0.031,live,0.036,even,0.021,travel,0.029
5,come,0.021,ecosearch,0.017,update,0.063,record,0.047,pfizer,0.033,today,0.025,count,0.027,patient,0.029,think,0.021,way,0.027
6,still,0.021,need,0.015,expert,0.047,kill,0.039,make,0.025,disorder,0.023,supply,0.022,positive,0.026,ple,0.020,approve,0.023
7,month,0.020,time,0.013,staff,0.043,rise,0.026,life,0.024,continue,0.023,doctor,0.022,begin,0.024,important,0.020,war,0.022
8,learn,0.020,know,0.013,medical,0.034,daily,0.015,dose,0.022,major,0.022,here,0.022,develop,0.023,show,0.019,hard,0.021
9,last,0.019,work,0.012,early,0.034,man,0.012,spread,0.018,reduction,0.017,put,0.022,warn,0.020,science,0.019,provide,0.021
